<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/140_Langchain_Intro_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# 🔹 What is LangChain?

* LangChain is a **Python (and JS) framework** designed for building applications powered by large language models (LLMs).
* Instead of you wiring every agent and status check manually, LangChain gives you **pre-built abstractions** for:

  * **Chains** → sequences of calls (like functions or agents in a row).
  * **Agents** → LLMs that can decide which tools to call.
  * **Tools** → wrappers around external functions, APIs, or databases.
  * **Memory** → persistence of context across turns.
  * **Callbacks / Tracing** → built-in observability, logging, and monitoring.
  * **Runnables** → composable building blocks for pipelines.

Think of LangChain as an **agent/pipeline framework** the way Django is a framework for web apps: it saves you from reinventing the plumbing.

---

# 🔹 Benefits of Using LangChain for an Orchestrator

### 1. **Less Boilerplate**

* Your custom orchestrator explicitly defines workflow steps, state tracking, retries, etc.
* LangChain provides **Chains** and **Sequential/Parallel executors** out of the box, so you don’t have to reinvent that control flow.

---

### 2. **Built-in Agent Capabilities**

* LangChain agents already know how to:

  * Use an LLM to decide which tool to call.
  * Keep looping until a condition is met.
  * Handle “thought → action → observation → repeat” loops (ReAct pattern).
* In your custom orchestrator, you coded these decisions manually. LangChain gives you scaffolding.

---

### 3. **Tool & Integration Ecosystem**

* 100+ prebuilt connectors (APIs, databases, web search, CRMs).
* Instead of writing wrappers for LinkedIn, CRM, or email APIs yourself, you can reuse community-tested LangChain tools.
* Speeds up prototyping dramatically.

---

### 4. **Memory + Context Persistence**

* Your orchestrator tracks state with enums and dataclasses.
* LangChain has **Memory modules** to carry context across steps or conversations automatically.
* That’s useful if you want multi-session or human-in-the-loop review with history intact.

---

### 5. **Observability**

* LangSmith (companion tool) gives you **logs, traces, metrics, and dashboards** without writing custom monitoring.
* In your demo pipeline, you printed out metrics manually — LangChain gives you observability as a first-class citizen.

---

### 6. **Composability**

* LangChain uses the `Runnable` interface, so you can treat agents, chains, or even a whole pipeline as **lego blocks**.
* Swap out your ResearchAgent for a new one without breaking the orchestrator.
* In your current hand-built orchestrator, you had to code those connections manually.

---

# 🔹 Trade-offs vs. Hand-Rolled Orchestrator

* **Pros of LangChain**: faster dev time, built-in patterns (ReAct, Tools, Chains), big ecosystem, observability.
* **Cons**: extra dependency, opinionated abstractions, can feel “heavy” if you just want a simple pipeline, less transparency if you need fine-grained control.

Your **custom orchestrator** = maximum control, explicit state, excellent for learning how orchestration really works.
**LangChain orchestrator** = productivity booster, batteries included, lets you focus on business logic rather than plumbing.

---

✅ **Key takeaway:**
LangChain won’t do anything magical that you couldn’t code yourself (you already did!). But it **standardizes best practices**, saves time, and plugs you into a community ecosystem of tools and patterns.




## 1. Less Boilerplate
Your custom orchestrator explicitly defines workflow steps, state tracking, retries, etc.
LangChain provides Chains and Sequential/Parallel executors out of the box, so you don’t have to reinvent that control flow.




### 🔹 1. Hand-Rolled Orchestrator (Your Style)

Here’s a simplified version of what you’ve already built. Notice how **you have to manage the control flow, retries, and state explicitly**:


✅ **What’s happening:**

* You define steps, status, retries.
* You manually wire step execution (`if research_result: ...`).
* Full control, but lots of **plumbing code**.



In [ ]:
from dataclasses import dataclass
from enum import Enum
import time

class AgentStatus(Enum):
    PENDING = "PENDING"
    RUNNING = "RUNNING"
    COMPLETED = "COMPLETED"
    FAILED = "FAILED"

@dataclass
class WorkflowStep:
    name: str
    status: AgentStatus = AgentStatus.PENDING
    retries: int = 0
    max_retries: int = 3
    output: str = None

def run_step(step: WorkflowStep, func, *args, **kwargs):
    while step.retries < step.max_retries:
        step.status = AgentStatus.RUNNING
        try:
            result = func(*args, **kwargs)
            step.output = result
            step.status = AgentStatus.COMPLETED
            return result
        except Exception as e:
            step.retries += 1
            print(f"{step.name} failed ({step.retries} retries). Error: {e}")
            time.sleep(1)
    step.status = AgentStatus.FAILED
    return None

# Example workflow
def research(company): return f"Research on {company}"
def analyze(data): return f"Analysis of {data}"

steps = [
    WorkflowStep(name="Research"),
    WorkflowStep(name="Analysis")
]

research_result = run_step(steps[0], research, "Acme Corp")
if research_result:
    analyze_result = run_step(steps[1], analyze, research_result)

print(steps)



### 🔹 2. LangChain Equivalent

In LangChain, you **don’t write retry/state machinery yourself**. You just define the steps (functions, LLMs, tools), then chain them.

✅ **What’s happening here:**

* `RunnableSequence` handles the **pipeline execution** automatically.
* You don’t write retry/status boilerplate — LangChain provides hooks for retries if you want them.
* Inputs/outputs automatically flow from one step to the next.

---

# 🔹 Side-by-Side Insight

* **Your code**: \~40 lines for 2 steps, manual state management, retry loops.
* **LangChain**: \~10 lines for 2 steps, declarative chaining.




In [ ]:
from langchain_core.runnables import RunnableSequence

# Define simple functions
def research(company: str) -> str:
    return f"Research on {company}"

def analyze(data: str) -> str:
    return f"Analysis of {data}"

# Build a chain (step 1 → step 2)
pipeline = RunnableSequence(first=research, last=analyze)

# Run it
result = pipeline.invoke("Acme Corp")
print(result)


# 🔹 What is `RunnableSequence`?

`RunnableSequence` is a **LangChain primitive** that lets you define a sequence (pipeline) of steps where:

* The **output of one step** becomes the **input of the next step**.
* You can pass in **functions, chains, agents, or even LLMs** as steps.
* It automatically manages the flow — you don’t write the `result = step1(); step2(result)` boilerplate.

It’s part of the **Runnable interface** in LangChain, which is their universal abstraction for “things you can run.”

---

# 🔹 Example: Simple Functions

```python
from langchain_core.runnables import RunnableSequence

def step1(x: str) -> str:
    return x.upper()

def step2(y: str) -> str:
    return f"Hello, {y}!"

pipeline = RunnableSequence(first=step1, last=step2)

print(pipeline.invoke("world"))
# Output: "Hello, WORLD!"
```

👉 Instead of you manually writing:

```python
result1 = step1("world")
result2 = step2(result1)
```

LangChain wires them together.

---

# 🔹 What makes it powerful?

1. **Uniform interface**

   * Every `Runnable` supports:

     * `.invoke(input)` → run once, sync.
     * `.batch([inputs])` → run on a list.
     * `.stream(input)` → get outputs incrementally (great for streaming LLM tokens).

   This means you can swap a function with an LLM, and the pipeline still works.

---

2. **Composability**

   * You can nest them:

   ```python
   pipeline = step1 | step2 | step3
   ```

   The `|` operator is syntactic sugar for `RunnableSequence`.

---

3. **Flexibility**

   * Steps can be:

     * Python functions
     * LLM calls
     * Chains
     * Agents
     * Other pipelines

   So you can mix and match: a research function → an LLM summarizer → a formatting function.

---

# 🔹 Why use `RunnableSequence`?

It solves the **“less boilerplate”** problem:

* Instead of writing explicit loops, conditionals, and variable passing,
* You declare a **pipeline graph**, and LangChain executes it.

Think of it as a **conveyor belt**: put something at the start, and it automatically passes through each machine until you get the finished product.

---

✅ **Key takeaway:**
`RunnableSequence` = LangChain’s way of wiring steps together declaratively. It reduces boilerplate and makes your pipeline flexible, swappable, and consistent.




# 2. Built-in Agent Capabilities



## 🔹 1. Hand-Rolled Orchestrator (Your Style)

In your orchestrator, if you want an agent to decide which tool to use, you have to:

* Define all tools yourself.
* Manually write the control flow (`if … elif … else`).
* Manage retries, context, and state.

Example (simplified):

```python
def search_web(query: str) -> str:
    return f"Web results for {query}"

def lookup_crm(company: str) -> str:
    return f"CRM data for {company}"

def custom_agent(task: str, company: str):
    if "search" in task.lower():
        return search_web(company)
    elif "crm" in task.lower():
        return lookup_crm(company)
    else:
        return "No tool available"

print(custom_agent("search info", "Acme Corp"))
# → "Web results for Acme Corp"
```

✅ This works, but **you’re the one coding the decision tree**.
The LLM isn’t “choosing” — you are.

---



## 🔹 2. LangChain Agent (Built-In)

LangChain flips this:

* You give the agent a **set of tools**.
* You give it an **LLM**.
* The agent uses the **ReAct pattern** (Thought → Action → Observation → Repeat) to decide which tool to call, in what order, until the task is complete.

Example:

```python
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, Tool

# Define tools
def search_web(query: str) -> str:
    return f"Web results for {query}"

def lookup_crm(company: str) -> str:
    return f"CRM data for {company}"

tools = [
    Tool(
        name="WebSearch",
        func=search_web,
        description="Use this to search the web for company information"
    ),
    Tool(
        name="CRM",
        func=lookup_crm,
        description="Use this to look up CRM data about a company"
    )
]

# Define LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Create agent with tools
agent = initialize_agent(
    tools,
    llm,
    agent="zero-shot-react-description",
    verbose=True
)

# Ask the agent
result = agent.run("Get background info about Acme Corp from CRM")
print(result)
```

🪄 What happens behind the scenes:

1. The LLM **thinks**: “I need CRM data, not web search.”
2. It chooses the **CRM tool**.
3. Executes `lookup_crm("Acme Corp")`.
4. Gets the result.
5. Returns the final answer.

You didn’t write `if CRM else Web` logic — the **LLM + LangChain agent did**.

---

# 🔹 Key Benefits

1. **Dynamic decision-making**

   * The agent can flexibly decide which tool(s) to call.
   * You don’t hardcode conditions.

2. **ReAct loop**

   * Thought → Action → Observation → Repeat
   * Lets the agent do multi-step reasoning automatically.

3. **Extensible**

   * Add more tools (e.g., LinkedIn API, Email Writer).
   * The agent can figure out which to use without rewriting your orchestrator.

---

✅ **Key takeaway:**
In your custom orchestrator, *you* are the brain coding the tool-selection logic.
In LangChain, the **LLM is the brain**, and the agent framework gives it a safe sandbox to act in.





## 🔹 How Mental Overhead Works for LLMs

When you use an LLM raw (no framework, no structure):

* You have to stuff *everything* into the prompt: context, rules, tool definitions, output schema.
* The LLM has to parse a **giant wall of instructions** every time.
* This increases **cognitive load** → more hallucinations, less reliable tool use, higher token cost.

---

# 🔹 What LangChain Agents Do Differently

LangChain **reduces this load** in a few ways:

### 1. **Standardized Tool Schema**

* Instead of you writing long prompt instructions like:
  *“If user asks for CRM data, call this Python function with one string argument, else if they ask for web search, call this other function…”*
* LangChain just gives the LLM a clean **tool description** in JSON-like format.

The agent sees something like:

```json
{
  "name": "CRM",
  "description": "Use this to look up CRM data about a company",
  "args": { "company": "string" }
}
```

Much easier for the LLM to reason over. ✅

---

### 2. **The ReAct Loop (Thought → Action → Observation → Repeat)**

* Instead of handling everything in one giant completion,
* LangChain breaks reasoning into **small, iterative steps**:

  * *“I should check CRM first.”*
  * → Calls CRM tool.
  * *“Now I know their size, I should draft an outreach message.”*

This reduces “mental overhead” because the LLM doesn’t need to solve everything at once — it works step by step.

---

### 3. **Separation of Concerns**

* In your hand-rolled orchestrator, the LLM has to handle *both* reasoning and workflow control.
* In LangChain, the **orchestrator handles control flow**, retries, logging, etc.
* The LLM just focuses on: *“Which tool? What arguments? What’s next?”*

---

# 🔹 Trade-Offs

* ✅ Reduced overhead → fewer hallucinations, lower token usage, more reliable tool calls.
* ✅ Easier to scale → add more tools, LLM just “chooses” instead of parsing new giant prompts.
* ⚠️ Slight overhead on framework side → LangChain injects helper prompts/tool schemas into the context.
* ⚠️ Still requires **good tool descriptions** → if you describe tools vaguely, the LLM can misuse them.

---

✅ **Bottom line:**
Yes — LangChain’s agent framework **reduces LLM mental overhead** by giving it structured tool definitions, step-by-step reasoning loops, and offloading workflow control to the orchestrator.
The LLM can then focus on *problem-solving*, not *remembering your orchestration rules*.



## 3. Tool & Integration Ecosystem
100+ prebuilt connectors (APIs, databases, web search, CRMs).
Instead of writing wrappers for LinkedIn, CRM, or email APIs yourself, you can reuse community-tested LangChain tools.
Speeds up prototyping dramatically.

Perfect — this is where LangChain really shines 🌟.

You’ve already seen how you wrote your own wrappers (e.g. `search_web()`, `lookup_crm()`), which is fine for learning, but as soon as you scale into real-world pipelines, you’ll spend **tons of time just writing glue code**.

LangChain’s **tool ecosystem** solves that with **prebuilt integrations**.

---

# 🔹 1. Hand-Rolled Approach (Your Orchestrator)

Let’s say you want your agent to:

* Search Google,
* Query a SQL database,
* Send an email.

You’d have to manually write wrappers like:

```python
import requests
import sqlite3
import smtplib

def google_search(query: str) -> str:
    resp = requests.get("https://api.google.com/search", params={"q": query})
    return resp.json()

def query_db(sql: str) -> str:
    conn = sqlite3.connect("mydb.sqlite")
    cursor = conn.execute(sql)
    return cursor.fetchall()

def send_email(to, subject, body):
    with smtplib.SMTP("smtp.gmail.com") as server:
        server.sendmail("me@example.com", to, f"Subject: {subject}\n\n{body}")
```

✅ Works fine.
⚠️ But you’re reinventing the wheel, handling errors, API changes, authentication, retries, etc.

---

# 🔹 2. LangChain Tool Ecosystem (Built-In)

LangChain gives you **community-tested, production-ready connectors** for all of this.

Example:

```python
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, load_tools

# Load some tools out of the box
tools = load_tools(
    ["serpapi", "sql-database", "gmail"],
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0)
)

# Create agent with these tools
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

# Ask the agent something
agent.run("Search for the latest on Acme Corp, check my SQL database for their customer ID, then draft and email me the results.")
```

🪄 What happens:

* The **Google Search** tool runs via SerpAPI (no manual requests code).
* The **SQLDatabase** tool queries your DB (with SQLAlchemy integration).
* The **Gmail** tool sends email through your account.

You didn’t write a single wrapper — just imported tools.

---

# 🔹 Tool Categories You Get Out-of-the-Box

LangChain already has **100+ integrations** for:

* 🌐 Web search (SerpAPI, Tavily, Bing, DuckDuckGo)
* 📊 Databases (Postgres, SQL, MongoDB, Pinecone, Chroma, Weaviate)
* 📧 Email (Gmail, Outlook)
* 🔗 Productivity (Slack, Notion, Google Drive, Trello, Jira)
* 💳 Business tools (Salesforce CRM, HubSpot, Zendesk)
* 🧠 ML/Vector stores (FAISS, Milvus, Pinecone, Chroma)

So instead of writing glue code, you get **plug-and-play tools**.

---

# 🔹 Key Benefits

* ✅ **Faster prototyping** — build a working multi-tool agent in minutes.
* ✅ **Less maintenance** — community maintains connectors.
* ✅ **Consistency** — every tool follows the same interface.
* ✅ **Extensibility** — you can still add custom tools if needed.

---

✅ **Key takeaway:**
Your custom orchestrator requires **you** to wrap every API.
LangChain gives you **a huge library of tools**, ready to use with agents — speeding up prototyping and reducing boilerplate.



## mini sales pipeline agent

Let’s wire up a **mini sales pipeline agent** using LangChain’s built-in tools 🚀.
This will look *very* similar to your orchestrator, but with far less glue code.

---

# 🔹 Step 1. Import LLM & Agent

```python
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, load_tools

# Base LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
```

---

# 🔹 Step 2. Load Tools

Instead of writing wrappers, we just load them:

```python
# Load prebuilt tools
tools = load_tools(
    ["linkedin", "gmail", "hubspot"],   # all available in LangChain ecosystem
    llm=llm
)
```

What you get:

* **LinkedIn Tool** → Search profiles, pull company/role info.
* **Gmail Tool** → Draft & send emails.
* **HubSpot Tool** → Query CRM data about leads/companies.

---

# 🔹 Step 3. Create the Agent

```python
# Create agent with ReAct reasoning loop
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)
```

---

# 🔹 Step 4. Run a Sales Task

Now let’s run something similar to your workflow:

```python
result = agent.run(
    "Look up Acme Corp on LinkedIn, check HubSpot for existing contacts, "
    "then draft a personalized outreach email and send it via Gmail."
)

print(result)
```

---

# 🔹 🧠 What Happens Behind the Scenes

1. Agent thinks: *“I need company info → use LinkedIn tool.”*

   * Action: `LinkedIn.search("Acme Corp")`
   * Observation: Profile data returned.

2. Agent thinks: *“Now check CRM to see if we already have contacts.”*

   * Action: `HubSpot.query("Acme Corp")`
   * Observation: Contact found: *Jane Doe, Head of Ops.*

3. Agent thinks: *“Draft outreach message for Jane Doe.”*

   * Calls LLM internally to craft the email.

4. Agent thinks: *“Send email via Gmail.”*

   * Action: `Gmail.send(to="jane.doe@acme.com", subject="Intro", body="…")`

5. Final Answer:

   * *“Outreach email successfully sent to Jane Doe at Acme Corp.”*

---

# 🔹 Why This Is Huge

* ⚡ **Zero glue code** → you didn’t write any API calls.
* 🪄 **LLM decides flow** → you didn’t write `if/else` logic.
* 🔌 **Pluggable** → swap HubSpot for Salesforce by just changing `"hubspot"` → `"salesforce"`.
* 🧱 **Scalable** → add more tools (Slack, Notion, Calendars) and the agent can use them without refactoring.

---

✅ **Key takeaway:**
Your orchestrator = you do the wiring.
LangChain agent = you declare the tools, and the LLM figures out how to chain them.




LangChain itself doesn’t ship a single official “LinkedIn profile tool” with a guaranteed spec — many integrations are third-party, and their outputs vary. But there *are* tools in LangChain’s ecosystem (or via integrations like BrightData, Proxycurl, etc.) that target LinkedIn person or company profiles, and I can walk you through what kinds of info they often return, what to watch out for, and what assumptions to make.

---

## 🔹 What “LinkedIn tool” might return — common fields

If you use a LinkedIn profile scraping tool, or a “LinkedIn person profile” dataset via something like BrightData’s Web Scraper API, typical fields include:

| Field                                   | What it means / Examples                                                                         |
| --------------------------------------- | ------------------------------------------------------------------------------------------------ |
| `name`                                  | Full name of the person (e.g. “Jane Doe”)                                                        |
| `headline` or `title`                   | Their role / what they list as their professional title (e.g. “Head of Engineering at StartupX”) |
| `company` or `current_company`          | The company they are currently working at                                                        |
| `industry`                              | The industry of that company (if available)                                                      |
| `location`                              | City, region, or country (depending on availability)                                             |
| `education`                             | Their education history (e.g. schools, degrees)                                                  |
| `experience`                            | Past roles + companies, sometimes durations or dates                                             |
| `summary` or `bio`                      | Their “About” section or summary description                                                     |
| `skills`                                | A list of skills or endorsements (if the tool gets that level of detail)                         |
| `connections_count` or follower metrics | Number of connections/followers, etc., if exposed                                                |
| `contact_info`                          | Sometimes email or phone, if tool/API has access (this is often missing or requires premium)     |
| `profile_url`                           | The URL to the LinkedIn profile itself                                                           |
| `years_at_current_company` or `tenure`  | How long they’ve been with their current employer (if the data is scraped)                       |
| `past_positions`                        | Prior roles or jobs (titles & companies)                                                         |
| `company_size` / `employee_count`       | The size of the company (if the tool provides that via LinkedIn or company-profile data)         |
| `company_funding` / `company_website`   | For “company profile” tools, sometimes data about the firm (website, stage, etc.)                |

Some tools may also return more advanced/derived metrics:

* “Seniority score” — based on title (e.g. mapping “Manager”, “Director”, “VP”, “C-suite”)
* “Reachability” — e.g. does the profile have a public LinkedIn email, or is email found via enrichment
* “Profile completeness” (how much info is filled out)
* Recent activity / posts / leadership contributions

---

## 🔹 Example: BrightData’s Web Scraper API

For example, the **BrightDataWebScraperAPI** in LangChain (part of `langchain_brightdata`) supports a `dataset_type` of `"linkedin_person_profile"` or `"linkedin_company_profile"`. ([LangChain][1])

If you invoke it with a personal LinkedIn profile URL and `dataset_type="linkedin_person_profile"`, you might get many of the fields above, depending on what the scraper can access. For example:

```python
from langchain_brightdata import BrightDataWebScraperAPI

scraper = BrightDataWebScraperAPI(bright_data_api_key="YOUR_KEY")

result = scraper.invoke({
    "url": "https://www.linkedin.com/in/someperson/",
    "dataset_type": "linkedin_person_profile"
})

# result might be a dict like:
# {
#   "name": "Some Person",
#   "headline": "VP of Engineering at ExampleCo",
#   "current_company": "ExampleCo",
#   "location": "USA",
#   "experience": [
#     {"position": "VP Engineering", "company": "ExampleCo", "duration": "2 yrs"},
#     {"position": "Engineering Director", "company": "OldCo", "duration": "3 yrs"},
#     ...
#   ],
#   "education": [...],
#   "skills": [...],
#   "profile_url": "https://linkedin.com/in/someperson",
#   ...
# }
```

---

## 🔹 What to verify / what pitfalls to watch out for

Since output formats and reliability vary a lot, there are some “gotchas” you’ll want to be aware of:

1. **Legal / ethical / terms of service**

   * Scraping LinkedIn or using third-party tools might violate LinkedIn’s policies. Always check licensing / API usage and ensure you have permission.
   * If you use an official API, sometimes data fields are limited.

2. **Completeness & consistency**

   * Some fields might be missing. For instance, many LinkedIn profiles don’t show a public email or “years at current company”.
   * Titles and headline fields can be vague or stylized.

3. **Data freshness**

   * Profiles change. A tool might have stale data.
   * Some scrapers cache or delay updates.

4. **Reachability**

   * Just because you see a profile doesn’t mean you can contact via email. That contact info is often not public.

5. **Privacy / GDPR / compliance**

   * If contact info or personal identifiers are involved, ensure your usage meets data protection regulations.

6. **Rate limits / cost**

   * Many tools (e.g BrightData or Proxycurl) charge per profile or have rate limits. Be sure you know cost & throttling.

---

## 🔹 “Reliable + community-tested” reality check

* Yes, because many tools are build/tested by multiple users, with bug reports, etc., they tend to be more stable than one-off scrapers you write from scratch.
* But “community-tested” does *not* guarantee that the data fields you need exist in every instance, or that performance is always fast / error-free.

It still helps to treat them as “best effort”, validate their output, and build fallbacks (e.g., if “experience” is missing, don’t crash, skip or estimate).

---

If you like, I can find a few specific LinkedIn-tools in LangChain (names & documentation) and show exactly what each returns in their docs, so you know what to expect in your pipeline.

[1]: https://python.langchain.com/docs/integrations/tools/brightdata-webscraperapi/?utm_source=chatgpt.com "BrightDataWebScraperAPI"
